# **DataSalud: Exploración de Reseñas de Medicamentos y Análisis de Sentimientos**


# Análisis Exploratorio de los datos <a id='data_review'></a>


In [3]:
# Conectar Google Drive en el entorno de Google Colab para acceder a archivos y carpetas
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# se se importan las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import nltk
from nltk.corpus import stopwords

%matplotlib inline

In [27]:
# se guarda el dataset en un DataFrame
df = pd.read_csv('DrugReviews.csv', sep = ',')


In [28]:
# se muestran las 5 primeras filas
df.head(5)

,MedicineName,MedicineFor,ReviewDate,UserName,IntakeTime,Reviews,ReviewLength,Rating,NumberOfLikes
0,12 Hour Nasal Decongestant Spray,For Nasal Congestion,26-Jan-21,xano,Not Specified,This is very effective IF you can get the cove...,52,6,0
1,12 Hour Nasal Decongestant Spray,For Nasal Congestion,19-Aug-22,Breat...,Taken for 1 to 2 years,Actually I use the generic brand of the 12 hou...,319,10,0
2,12 Hour Nasal Decongestant Spray,For Nasal Congestion,28-Apr-18,Abe,Taken for less than 1 month,Cap took 20 minutes to open process was frustr...,373,1,0
3,5-HTP,For Anxiety,3-May-20,Andres,Taken for less than 1 month,Hi everyone\r\n'10 / 105-HTPFor Anxiety23129-O...,623,10,345
4,5-HTP,For Anxiety,11-Jul-19,Shawn,Not Specified,Took SSRI (Prozac) for Anxiety/Depression for ...,156,9,229


In [29]:
# se obtinen las dimensiones del DataFrame
df.shape

(392510, 9)

In [30]:
# se obtiene la información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392510 entries, 0 to 392509
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   MedicineName   392510 non-null  object
 1   MedicineFor    392510 non-null  object
 2   ReviewDate     392510 non-null  object
 3   UserName       392454 non-null  object
 4   IntakeTime     392510 non-null  object
 5   Reviews        392510 non-null  object
 6   ReviewLength   392510 non-null  int64 
 7   Rating         392510 non-null  int64 
 8   NumberOfLikes  392510 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 27.0+ MB


#  Limpieza y preprocesamiento de Datos

In [31]:
# se encuentran el número de valores ausentes
df.isna().sum().sort_values(ascending = False)

UserName         56
MedicineName      0
MedicineFor       0
ReviewDate        0
IntakeTime        0
Reviews           0
ReviewLength      0
Rating            0
NumberOfLikes     0
dtype: int64

In [32]:
# los valores ausentes se sustituyeb por Unknown
df['UserName'] = df['UserName'].fillna('Unknown')

In [33]:
# se verifican los valores ausentes
df.isna().sum().sort_values(ascending = False)

MedicineName     0
MedicineFor      0
ReviewDate       0
UserName         0
IntakeTime       0
Reviews          0
ReviewLength     0
Rating           0
NumberOfLikes    0
dtype: int64

# Distribución de los Datos

In [34]:
# se calculan los estadísticos descriptivos de las columnas ReviewLength, Rating y NumberOfLikes
df[['Rating', 'ReviewLength', 'NumberOfLikes']].describe()

,Rating,ReviewLength,NumberOfLikes
count,392510.000000,392510.000000,392510.000000
mean,6.281364,336.650422,26.995376
std,3.576652,262.251652,49.376236
min,1.000000,1.000000,0.000000
25%,2.000000,109.000000,4.000000
50%,8.000000,274.000000,13.000000
75%,10.000000,517.000000,32.000000
max,10.000000,4930.000000,3555.000000


In [35]:
# se calcula la moda de las columnas ReviewLength, Rating y NumberOfLikes
df[['Rating', 'ReviewLength', 'NumberOfLikes']].mode()

,Rating,ReviewLength,NumberOfLikes
0,10,38,0


In [36]:
# se grafica la distribución de las calificaciones de los medicamentos
fig1 = px.histogram(df,
                    x= 'Rating',
                    title= 'Distribución de las Calificaciones',
                    nbins= 20
                    )

fig1.update_xaxes(title_text= 'Calificación')
fig1.update_yaxes(title_text= 'Conteo',)

fig1.show()

<div style="background-color: lightyellow; padding: 10px;">

<span style="color: darkblue;">  
    
**Observaciones:**  
La distribución de las calificaciones no es normal, la moda de las calificaciones es de 10 y en promedio se tienen calificaciones de 6.2. Lo anterior se debe a que también hay una cantidad considerable de calificaciones de 0, lo que puede significar que no todos los usuarios quedaron satisfechos con el medicamento, por ello oorgaron calificaciones de 0.


</span>
    
</div>

In [37]:
# se grafica la distribución de Longitud de las reseñas
fig2 = px.histogram(df,
                    x= 'ReviewLength',
                    title= 'Distribución de la Longitud de las reseñas'
                    #nbins= 160
                    )

fig2.update_xaxes(title_text= 'Longitud de las Calificaciones')
fig2.update_yaxes(title_text= 'Conteo',)

fig2.show()

<div style="background-color: lightyellow; padding: 10px;">

<span style="color: darkblue;">  
    
**Observaciones:**  
Para la longitud de las reseñas su distribución tampoco es normal, la moda de la longitud es de 38 y en promedio los usuarios y usuarias hacen una reseña de 336 de longitud. En este caso hay una variabilidad mayor, ya que se tienen longitudes de 1 (valor mínimo) y 4930 (valor máximo).


</span>
    
</div>

In [38]:
# se grafica la distribución del número de likes
fig3 = px.histogram(df,
                    x= 'NumberOfLikes',
                    title= 'Distribución del Número de Likes'
                    )

fig3.update_xaxes(title_text= 'Número de Likes')
fig3.update_yaxes(title_text= 'Conteo',)

fig3.show()

<div style="background-color: lightyellow; padding: 10px;">

<span style="color: darkblue;">  
    
**Observaciones:**  
La distribución del número de likes tampoco es normal, la moda es de 0 y el promedio es de 26.9 likes. Lo anterior se puede deber a que no todos los usuarios dejan a dan like al medicamento que consumieron y por ello se presenta muchas variabilidad de los datos, ya que su valor mínimo es de 0 y su máximo de 3555.


</span>
    
</div>

In [39]:
# se contabilizan los 20 padecimientos más frecuentes
top_20_condition = df['MedicineFor'].value_counts().reset_index().head(20)

In [46]:
# Crear el gráfico de barras
fig = px.bar(top_20_condition,
             x='count',
             y='MedicineFor',
             orientation='h',  # Cambia a 'h' para orientación horizontal
             title='20 padecimientos más frecuentes',
             labels={'count': 'Cantidad', 'MedicineFor': 'Padecimiento'})

# Mostrar el gráfico
fig.show()

In [43]:
# Convertir la columna ReviewDate a datetime
df['ReviewDate'] = pd.to_datetime(df['ReviewDate'], format="%d-%b-%y")

# Crear nuevas columnas para año, mes y día
df['Year'] = df['ReviewDate'].dt.year
df['Month'] = df['ReviewDate'].dt.month
df['Day'] = df['ReviewDate'].dt.day

# Mostrar el DataFrame resultante
df.head(3)

,MedicineName,MedicineFor,ReviewDate,UserName,IntakeTime,Reviews,ReviewLength,Rating,NumberOfLikes,Year,Month,Day
0,12 Hour Nasal Decongestant Spray,For Nasal Congestion,2021-01-26,xano,Not Specified,This is very effective IF you can get the cove...,52,6,0,2021,1,26
1,12 Hour Nasal Decongestant Spray,For Nasal Congestion,2022-08-19,Breat...,Taken for 1 to 2 years,Actually I use the generic brand of the 12 hou...,319,10,0,2022,8,19
2,12 Hour Nasal Decongestant Spray,For Nasal Congestion,2018-04-28,Abe,Taken for less than 1 month,Cap took 20 minutes to open process was frustr...,373,1,0,2018,4,28


In [45]:
# Crear datos de ejemplo (reemplaza esto con tus datos reales)
anios_conteo = df['Year'].value_counts()

# Crear el gráfico de barras
fig = px.bar(anios_conteo, x=anios_conteo.index, y=anios_conteo.values, # Use anios_conteo.values for the y-axis
             labels={'x': 'Año de reseña', 'y': 'Número de ocurrencias'}) # Update labels
fig.update_layout(title='Reseñas por año')

# Mostrar el gráfico
fig.show()


# Análisis de sentimientos: Efectos secundarios


In [49]:
# Descarga del toolkit
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [50]:
#Conjunto de stop words
stop_words = set(stopwords.words('english'))

In [51]:
#Limpieza de los reviews del dataset
df['Reviews'] = df['Reviews'].str.replace('[^\w\s]', '')
df['Reviews'] = df['Reviews'].str.replace('\d', '')
df['Reviews'] = df['Reviews'].str.replace('\\n', '')

Seleción de las 10 categorias más relevantes para el análisis.

In [7]:
# Lista de strings para filtrar
strings_to_filter = ['Birth Control', 'Depression', 'Anxiety', 'Weight Loss', 'Pain', 'Acne', 'Insomnia', 'Bipolar Disorder', 'Diabetes, Type 2', 'High Blood Pressure']

# DataFrame vacío para almacenar los resultados
df_filtered_combined = pd.DataFrame()

# Loop for para filtrar, agrupar y reemplazar el valor de la columna 'MedicineFor'
for string in strings_to_filter:
    df_filtered = df[df['MedicineFor'].str.contains(string)]
    df_filtered['MedicineFor'] = string  # Reemplazar el valor de la columna 'MedicineFor'
    df_filtered_combined = pd.concat([df_filtered_combined, df_filtered])

df = df_filtered_combined
df

<ipython-input-7-30b0aa39abf1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['MedicineFor'] = string  # Reemplazar el valor de la columna 'MedicineFor'


,MedicineName,MedicineFor,ReviewDate,UserName,IntakeTime,Reviews,ReviewLength,Rating,NumberOfLikes
2234,Apri,Birth Control,21-Dec-20,Victo...,Not Specified,I’ve been on this pill for a little over a yea...,86,10,1
2235,Apri,Birth Control,21-Nov-16,Marie...,Taken for 1 to 2 years,This birth control failed and I am due in May....,139,1,2
3372,Aviane,Birth Control,26-Nov-09,Anonymous,Not Specified,I've been on Aviane for about 2 weeks now. Jus...,82,8,2
3373,Aviane,Birth Control,27-May-11,KBirdy,Not Specified,Overall I like this pill but it makes me so em...,65,8,1
3374,Aviane,Birth Control,29-Jan-12,Anonymous,Not Specified,I was on it for a few weeks and started to get...,193,1,1
...,...,...,...,...,...,...,...,...,...
383614,Ziac,High Blood Pressure,11-Dec-22,rick,Taken for 10 years or more,Been taking this for 15 years or so started at...,70,7,0
383615,Ziac,High Blood Pressure,24-Jun-08,Anonymous,Not Specified,Very good medication; no complications.,39,7,29
383616,Ziac,High Blood Pressure,18-Oct-09,Aljazi,Not Specified,It's been good for me. I never reacted well to...,185,7,20
383617,Ziac,High Blood Pressure,28-Aug-12,Anonymous,Taken for less than 1 month,Works great. Calming.,21,8,10


In [10]:
# Definición de los efectos secundarios a explorar
efectos_secundarios = ['nausea', 'headache', 'dizziness', 'fatigue','drowsiness','diarrhea']

In [12]:
# Función para tokenizar y filtrar reseñas
def encontrar_efectos_secundarios(texto):
    palabras = nltk.word_tokenize(texto.lower())
    palabras_filtradas = [palabra for palabra in palabras if palabra not in nltk.corpus.stopwords.words('english')]
    efectos_encontrados = [efecto for efecto in efectos_secundarios if efecto in palabras_filtradas]
    return efectos_encontrados


In [16]:
df['efectos_secundarios'] = df['Reviews'].apply(encontrar_efectos_secundarios)

In [17]:
# Explode the 'efectos_secundarios' column to create rows for each effect
df_exploded = df.explode('efectos_secundarios')

efectos_por_categoria = df_exploded.groupby(['MedicineFor', 'efectos_secundarios']).size().unstack(fill_value=0)

In [18]:
df_exploded.head()

,MedicineName,MedicineFor,ReviewDate,UserName,IntakeTime,Reviews,ReviewLength,Rating,NumberOfLikes,efectos_secundarios
2234,Apri,Birth Control,21-Dec-20,Victo...,Not Specified,I’ve been on this pill for a little over a yea...,86,10,1,NaN
2235,Apri,Birth Control,21-Nov-16,Marie...,Taken for 1 to 2 years,This birth control failed and I am due in May....,139,1,2,NaN
3372,Aviane,Birth Control,26-Nov-09,Anonymous,Not Specified,I've been on Aviane for about 2 weeks now. Jus...,82,8,2,NaN
3373,Aviane,Birth Control,27-May-11,KBirdy,Not Specified,Overall I like this pill but it makes me so em...,65,8,1,NaN
3374,Aviane,Birth Control,29-Jan-12,Anonymous,Not Specified,I was on it for a few weeks and started to get...,193,1,1,NaN


In [19]:
efectos_por_categoria

efectos_secundarios,diarrhea,dizziness,drowsiness,fatigue,headache,nausea
MedicineFor,,,,,,
Acne,20,46,3,47,41,246
Anxiety,279,475,220,260,394,997
Bipolar Disorder,33,92,84,80,73,261
Birth Control,299,559,18,902,793,4266
Depression,358,303,106,277,327,1084
"Diabetes, Type 2",670,106,5,155,197,1123
High Blood Pressure,76,291,15,176,146,127
Insomnia,22,58,111,46,173,64
Pain,91,170,101,134,352,447


In [25]:
efectos_por_categoria.to_csv('efectos_por_categoria.csv', index=False) # Guarda el DataFrame en un archivo CSV


In [21]:
# Reset the index to make 'MedicineFor' a column
efectos_por_categoria = efectos_por_categoria.reset_index()

In [22]:
# Crear el gráfico de barras agrupadas con Plotly
fig = px.bar(efectos_por_categoria, x='MedicineFor', y=['nausea', 'headache', 'dizziness', 'fatigue','drowsiness','diarrhea'],
             title='Efectos secundarios por categoría de medicamentos',
             labels={'value': 'Frecuencia de efectos secundarios', 'variable': 'Efectos Secundarios'},
             height=500)

# Rotar las etiquetas del eje x
fig.update_xaxes(tickangle=45)

# Mostrar el gráfico
fig.show()

In [23]:
# Crear el gráfico de barras agrupadas con Plotly
fig = px.bar(efectos_por_categoria, x='MedicineFor', y=['nausea', 'headache', 'dizziness', 'fatigue','drowsiness','diarrhea'],
             title='Efectos secundarios por categoría de medicamentos',
             labels={'value': 'Frecuencia de efectos secundarios', 'variable': 'Efectos Secundarios'},
             height=500, barmode='group')

# Rotar las etiquetas del eje x
fig.update_xaxes(tickangle=45)

# Mostrar el gráfico
fig.show()

In [26]:
# Preparar los datos para el mapa de calor
df_melted = efectos_por_categoria.melt(id_vars='MedicineFor', var_name='EfectoSecundario', value_name='Frecuencia')

# Crear el mapa de calor
fig = px.imshow(df_melted.pivot(index='MedicineFor', columns='EfectoSecundario', values='Frecuencia'),
                labels=dict(x="Side Effects", y="Drug Category", color="Frequency"),
                x=['diarrhea', 'dizziness', 'drowsiness', 'fatigue', 'headache', 'nausea'],
                y=df_melted['MedicineFor'].unique())

fig.show()
